In [1]:
import hw1_helpers as hh
import math

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots



from pint import UnitRegistry

## constants

- source: fluid properties https://www.climate-policy-watcher.org/wastewater-sludge/physical-and-biological-properties.html

In [2]:
# example constants 
ex_c = hh.constants(1, 1,  1, 1, 1, override=True)
ex_c

{'rho': 998.0 <Unit('kilogram / meter ** 3')>,
 'mu': 0.00102 <Unit('kilogram / meter / second')>,
 'D': 0.06 <Unit('meter')>,
 'h': 1 <Unit('meter')>,
 'beta': 1 <Unit('meter')>,
 'H': 2.2 <Unit('meter')>,
 'x ': 66.66666666666667 <Unit('meter')>,
 'theta': 89.14062775635531 <Unit('degree')>,
 'L': 65 <Unit('meter')>,
 'g': 9.8 <Unit('meter / second ** 2')>,
 'alpha': 2}

In [3]:
# real constants 
# density of primary sludge => 1.0 -> 1.03 g/cm3 =>=> 1000 kg/m3 -> 1030 kg/m3
# dynamic viscosity => 0.5 and 2 Pa*s  ~ kg/(m*s)
# worst case material scenario
rc = hh.constants(rho=1030, mu=2, D=0.1, h=1.1, beta=1.1 )
rc

{'rho': 1030.0 <Unit('kilogram / meter ** 3')>,
 'mu': 0.5 <Unit('kilogram / meter / second')>,
 'D': 0.1 <Unit('meter')>,
 'h': 1.1 <Unit('meter')>,
 'beta': 1.1 <Unit('meter')>,
 'H': 2.2 <Unit('meter')>,
 'x ': 73.33333333333334 <Unit('meter')>,
 'theta': 89.14062775635531 <Unit('degree')>,
 'L': 73.34158286932303 <Unit('meter')>,
 'g': 9.8 <Unit('meter / second ** 2')>,
 'alpha': 2}

# calculations

In [336]:
# set constants 
d = rc

In [ ]:
def calc_properties(d): 
    """calculates properties of the system given dictionary d of appropriate constants """

    # coefficients
    a = 1
    b = (64 * d["mu"] * d["L"] ) / (d["rho"] * d["D"]**2 * d["alpha"] )
    c =  -(2 * d["g"] * d["H"]) / (d["alpha"])

    coeff = {
        "a": a,
        "b": b,
        "c": c
    }

    print(coeff)

    # calculate velocity of fluid coming down first incline 
    vel_result = hh.solve_quad(a, b, c)

    return vel_result
    
    # vel = vel_result[0]

In [337]:
a = 1
b = (64 * d["mu"] * d["L"] ) / (d["rho"] * d["D"]**2 * d["alpha"] )
c =  -(2 * d["g"] * d["H"]) / (d["alpha"])

coeff = {
    "a": a,
    "b": b,
    "c": c
}

print(coeff)

{'a': 1, 'b': <Quantity(113.928672, 'meter / second')>, 'c': <Quantity(-21.56, 'meter ** 2 / second ** 2')>}


In [ ]:
# want to know how changing h and d affect the velocity => , currently have velocity of 0.00148 m / s, and need 40 m/s at least 

## plot

In [338]:
x =  np.linspace(-100, 100, 100)  * m / s

y = (a * x**2) + (b * x) + c

y[0]


-1414.427241836584 <Unit('meter ** 2 / second ** 2')>

In [339]:

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x.magnitude,
    y=y.magnitude, 
    mode='lines+markers',
))

fig.update_layout(title='Examine Quad - Solving for V',
                   xaxis_title='x ',
                   yaxis_title='y')

In [340]:
def solve_quad(a, b, c):
    # a = a
    # b = b.magnitude
    # c = c.magnitude
    print(a,b,c)
    d = b ** 2 - 4 * a * c  # this part is called the discriminant

    if d < 0:
        print("The equation has no real solutions")
    elif d == 0:
        x = (-b + np.sqrt(b ** 2 - 4 * a * c)) / (2 * a)
        print(f"The equation has one solution: {x} ")
        return x
    else:
        x1 = (-b + np.sqrt(b ** 2 - 4 * a * c)) / (2 * a)
        x2 = (-b - np.sqrt(b ** 2 - 4 * a * c)) / (2 * a)
        print(f"The equation has two solutions: {x1} or {x2}")
        return (x1,x2)
    

In [341]:
vel_result = solve_quad(a, b, c)

vel_result 

1 113.92867241836585 meter / second -21.560000000000002 meter ** 2 / second ** 2
The equation has two solutions: 0.18892791241233908 meter / second or -114.11760033077819 meter / second


(0.18892791241233908 <Unit('meter / second')>,
 -114.11760033077819 <Unit('meter / second')>)

In [342]:
vel = vel_result[0]
vel

0.18892791241233908 <Unit('meter / second')>

## volume flow rate

In [343]:
# # vfr , volume flow rate, m3/s
vfr = (vel* math.pi * d["D"]**2) / 4
vfr

0.0014838363542316513 <Unit('meter ** 3 / second')>

## work from the pump 

In [344]:
Re = (d["rho"]*vel*d["D"]) / d["mu"]
Re

38.91914995694185 <Unit('dimensionless')>

In [345]:
f = 64/ Re
f

1.6444346824328464 <Unit('dimensionless')>

In [346]:
vel**2

0.03569375608848447 <Unit('meter ** 2 / second ** 2')>

In [347]:
h_l = f * (d["L"] / d["D"]) *  0.5*((vel**2) / d["g"])
h_l

2.1963577799909286 <Unit('meter')>

In [348]:
# rate of pump work -> kg m2/s^3 (Watts)
w_pump = vfr * d["rho"] * d["g"] * h_l
w_pump

32.89670454746054 <Unit('kilogram * meter ** 2 / second ** 3')>

# boundaries


In [349]:
# 4 million m^3 / day => s <= minimum flow rate ?
city_fr = (4e6 * m**3 / u.day).to("m**3 / s")
city_fr
# 40 km of movement 
# 2.5 kw - 13.5 kW pumps 

46.29629629629629 <Unit('meter ** 3 / second')>

In [350]:
46

46